# Section D3 - Graphing with Pandas (and Matplotlib)

Feedback: https://forms.gle/Le3RAsMEcYqEyswEA

We'll go through a few types of graphs here!

## Time series

## Multi Axis

## Scatter Plot

## Box and Whisker Plot

## Histogram

## Heatmap

## Multiple Plots

In [ ]:
import pandas as pd
data_url = 'https://waterservices.usgs.gov/nwis/iv/?sites=11455485&startDT=2024-03-15T20:55:29.967-07:00&endDT=2024-10-15T20:55:29.967-07:00&format=rdb'
tol = pd.read_csv(data_url, sep='\t', comment='#', header=0)
tol.drop(tol.index[0], inplace=True)
print(tol)
# regression/scatter of specific conductivity and salinity
# time series with nitrate and chlorophyll; ph,do and chlorophyll...
# Boxplot of daily temperature